In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import numpy as np
import soundfile as sf
import torchaudio
import torchcodec

from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [ ]:
# -------------------------------------------------
# Device configuration
# -------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# -------------------------------------------------
# Load pretrained Wav2Vec2 model
# -------------------------------------------------
AUDIO_MODEL_NAME = "facebook/wav2vec2-base"
processor = Wav2Vec2Processor.from_pretrained(AUDIO_MODEL_NAME)
model = Wav2Vec2Model.from_pretrained(AUDIO_MODEL_NAME).to(DEVICE)
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [ ]:
TARGET_SR = 16000

def load_and_resample_audio(path):
    waveform, sr = torchaudio.load(path)   # waveform: [channels, samples]

    if sr != TARGET_SR:
        resampler = torchaudio.transforms.Resample(
            orig_freq=sr,
            new_freq=TARGET_SR
        )
        waveform = resampler(waveform)

    # convert to mono if needed
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0)

    return waveform.squeeze(), TARGET_SR


In [ ]:
AUDIO_FEATURE_PATH = "/content/drive/MyDrive/Dissertion/Data/Features/audio"

if not os.path.exists(AUDIO_FEATURE_PATH):
    os.makedirs(AUDIO_FEATURE_PATH)
# -------------------------------------------------
# Audio embedding extraction + saving
# -------------------------------------------------
@torch.no_grad()
def extract_and_save_audio_embedding(
    audio_path,
    save_dir=AUDIO_FEATURE_PATH
):
    audio_name = os.path.splitext(os.path.basename(audio_path))[0]


    load_and_resample_audio(audio_path)
   # Load WAV using soundfile
    waveform, sr = sf.read(audio_path, dtype="float32")

    """# Convert to mono if stereo
    if waveform.ndim == 2:
        waveform = waveform.mean(axis=1)

    waveform = torch.from_numpy(waveform)"""

    waveform, sr = load_and_resample_audio(audio_path)

    # Prepare input for Wav2Vec2
    inputs = processor(
        waveform,
        sampling_rate=sr,
        return_tensors="pt"
    ).to(DEVICE)

    # Forward pass through Wav2Vec2
    outputs = model(**inputs)

    # Frame-level embeddings: [time_steps, hidden_dim]
    hidden_states = outputs.last_hidden_state.squeeze(0)

    # -------------------------------------------------
    # ONE audio embedding per audio file (temporal mean)
    # -------------------------------------------------
    audio_embedding = hidden_states.mean(dim=0)  # [768]

    # L2 normalization
    audio_embedding = audio_embedding / audio_embedding.norm()

    #Save embedding
    save_path = os.path.join(save_dir, f"{audio_name}.pt")
    torch.save(audio_embedding.cpu(), save_path)
    #return audio_embedding.cpu(), save_path

In [ ]:
RAVDESS_DATA_PATH = "/content/drive/MyDrive/Dissertion/Data/Only_audio"

file_list = []
for root, dirs, files in os.walk(RAVDESS_DATA_PATH):

  for file in files:
        file_list.append(os.path.join(root, file))

print(f"total files found {len(file_list)}")
print(file_list[:5])

total files found 2408
['/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-02-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-01-01-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-01-02-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-04-01-02-01-22.wav', '/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-01-01-02-22.wav']


In [ ]:
total_files = len(file_list)
cnt = 0



for vis_file in file_list:
    audio_file = vis_file
    if not os.path.exists(audio_file):
        print(f"Warning: audio file not found: {audio_file}. Skipping.")
        continue
    try:
        extract_and_save_audio_embedding(audio_file)
        cnt += 1
        print(f"{cnt}/{total_files}  Embeddings generated for file:{audio_file}")
    except ValueError as e:
        print(f"Error processing {audio_file}: {e}. Skipping.")

1/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-02-01-22.wav
2/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-01-01-01-22.wav
3/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-01-01-01-02-22.wav
4/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-04-01-02-01-22.wav
5/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-01-01-02-22.wav
6/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-03-02-02-01-22.wav
7/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/03-02-02-02-02-01-22.wav
8/2408  Embeddings generated for file:/content/drive/MyDrive/Dissertion/Data/Only_audio/songs/Actor_22/0

In [ ]:
#load embeddings from feature path and show properties of them
import torch
from pathlib import Path

def inspect_embeddings(feature_dir):
    #print total files
    print(f"Total files: {len(list(Path(feature_dir).rglob('*.pt')))}")
    files = sorted(Path(feature_dir).rglob("*.pt"))[:10]
    print(f"Showing {len(files)} files from: {feature_dir}\n")

    for i, f in enumerate(files, 1):
        obj = torch.load(f)

        if isinstance(obj, dict):
            emb = obj.get("embedding")
            extra = f", keys={list(obj.keys())}"
        else:
            emb = obj
            extra = ""

        if emb is None:
            print(f"{i}. {f.name} -> EMPTY{extra}")
            continue

        shape = tuple(emb.shape)
        dtype = emb.dtype
        norm = float(emb.norm(p=2)) if emb.numel() > 0 else 0.0

        print(f"{i}. {f.name}")
        print(f"   path : {f}")
        print(f"   shape: {shape}")
        print(f"   dtype: {dtype}")
        print(f"   L2   : {norm:.4f}{extra}\n")

In [ ]:
inspect_embeddings(AUDIO_FEATURE_PATH)

Total files: 3024
Showing 10 files from: /content/drive/MyDrive/Dissertion/Data/Features/audio

1. 01-02-01-01-01-01-01.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/audio/01-02-01-01-01-01-01.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

2. 01-02-01-01-01-01-02.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/audio/01-02-01-01-01-01-02.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

3. 01-02-01-01-01-01-03.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/audio/01-02-01-01-01-01-03.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

4. 01-02-01-01-01-01-04.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/audio/01-02-01-01-01-01-04.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

5. 01-02-01-01-01-01-05.pt
   path : /content/drive/MyDrive/Dissertion/Data/Features/audio/01-02-01-01-01-01-05.pt
   shape: (768,)
   dtype: torch.float32
   L2   : 1.0000

6. 01-02-01-01-01-01-06.pt
   path